In [ ]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.7 MB/s eta 0:00:00


In [ ]:
datasets = {
    "Roboflow": [
        {
            "workspace": "yakand",
            "project": "lpr-cckha",
            "version": 1,
            "format": [
                "yolov8",
                "yolov5"
            ],
            "unwanted_classes": [
                3,
                5,
                6,
                7,
                9,
                10,
                11,
                14,
                19,
                20,
                24,
                25,
                27,
                28,
                29,
                31,
                35,
                36,
                39,
                41,
                42
            ],
            "class_map": {
                "0": 0,
                "1": 1,
                "2": 10,
                "3": 11,
                "4": 12,
                "5": 13,
                "6": 14,
                "7": 15,
                "8": 16,
                "9": 17,
                "10": 18,
                "11": 19,
                "12": 2,
                "13": 20,
                "14": 21,
                "15": 22,
                "16": 23,
                "17": 24,
                "18": 25,
                "19": 26,
                "20": 27,
                "21": 28,
                "22": 29,
                "23": 3,
                "24": 30,
                "25": 31,
                "26": 32,
                "27": 33,
                "28": 34,
                "29": 35,
                "30": 36,
                "31": 37,
                "32": 38,
                "33": 39,
                "34": 4,
                "35": 40,
                "36": 41,
                "37": 42,
                "38": 43,
                "39": 44,
                "40": 45,
                "41": 46,
                "42": 47,
                "43": 5,
                "44": 6,
                "45": 7,
                "46": 8,
                "47": 9
            }
        },
        {
            "workspace": "msc-9pkbx",
            "project": "new-zfexd",
            "version": 1,
            "format": [
                "yolov8",
                "yolov5"
            ],
            "unwanted_classes": [
                11,
                13,
                14,
                15,
                17,
                18,
                19,
                21,
                26,
                27,
                30,
                31,
                33,
                34,
                35,
                37,
                40,
                41,
                44,
                46,
                47
            ],
            "class_map": None
        }
    ]
}

In [ ]:
from roboflow import Roboflow
from google.colab import userdata

def download_roboflow_datasets(dataset, rf):
    """Download datasets from Roboflow to local directory."""
    for rf_dataset in datasets["Roboflow"]:  # iterate through Roboflow datasets
      project = rf.workspace(rf_dataset["workspace"]).project(rf_dataset["project"])
      version = project.version(rf_dataset["version"])
      dataset = version.download("yolov5", "./" + rf_dataset["project"] + "/")
      #process_classes("./" + rf_dataset["project"] + "/")

ROBOFLOW_API_KEY = userdata.get("ROBOFLOW_API_KEY")
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

download_roboflow_datasets(datasets, rf)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ./lpr-cckha/ in yolov5pytorch:: 100%|██████████| 18380/18380 [00:05<00:00, 3133.63it/s]


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ./new-zfexd/ in yolov5pytorch:: 100%|██████████| 20670/20670 [00:06<00:00, 3362.16it/s]


In [ ]:
import os

def process_classes(dataset_dir, unwanted_classes):
    """Filter out classes unrelated to Arabic OCR"""
    dirs = ["train/", "test/", "valid/"]

    for dir in dirs:
        path = dataset_dir + dir + "labels/"
        for label in os.listdir(path):
            with open(path + label, "r") as f:
                lines = f.readlines()
                for i in range(len(lines)):
                    class_id = int(lines[i].split(" ")[0])
                    if class_id in unwanted_classes:
                        lines[i] = ""

            with open(path + label, "w") as f:
                f.writelines(lines)

process_classes("./lpr-cckha/", datasets["Roboflow"][0]["unwanted_classes"])
process_classes("./new-zfexd/", datasets["Roboflow"][1]["unwanted_classes"])

In [ ]:
import os

def clean_dataset(dataset_dir):
    dirs = ["train/", "test/", "valid/"]
    for dir in dirs:
        path = dataset_dir + dir + "labels/"
        for label in os.listdir(path):
            f = open(path + label, "r")
            lines = f.readlines()
            f.close()
            if len(lines) == 0:
                os.remove(path + label)
                os.remove(dataset_dir + dir + "images/" + label[:-3] + "jpg")



clean_dataset("./lpr-cckha/")
clean_dataset("./new-zfexd/")

In [ ]:
import os
import shutil

def build_dirs(dataset):
    os.makedirs(dataset + "train/images/")
    os.makedirs(dataset + "train/labels/")

    os.makedirs(dataset + "test/images/")
    os.makedirs(dataset + "test/labels/")

    os.makedirs(dataset + "valid/images/")
    os.makedirs(dataset + "valid/labels/")


import random
import shutil

def compile_dataset(data):
    """Compile datasets from Roboflow into a single directory"""
    build_dirs("./arabic_compiled_dataset/")
    for dataset in data["Roboflow"]:
        dataset_dir = "./" + dataset["project"] + "/"
        dirs = ["train/", "test/", "valid/"]
        for dir in dirs:
            for image in os.listdir(dataset_dir + dir + "images/"):
                write = True
                label = image[:-3] + "txt"
                label_file = dataset_dir + dir + "labels/" + label
                if dataset["class_map"] is not None:
                    with open(label_file, "r") as label_f:
                        lines = label_f.readlines()
                        for i in range(len(lines)):
                            class_id = lines[i].split(" ")[0]
                            class_id = dataset["class_map"][class_id]
                            lines[i] = str(class_id) + " " + " ".join(lines[i].split(" ")[1:])
                            write = random.random() < 0.1 if str(class_id) == '35' else write
                    with open(label_file, "w") as label_f:
                        label_f.writelines(lines)
                if write:
                  shutil.copy(dataset_dir + dir + "images/" + image, "./arabic_compiled_dataset/" + dir + "images/")
                  shutil.copy(dataset_dir + dir + "labels/" + label, "./arabic_compiled_dataset/" + dir + "labels/")

compile_dataset(datasets)

In [ ]:
import cv2
import os

def remove_duplicates(dataset_dir):
    dirs = ["train/", "test/", "valid/"]
    for dir in dirs:
        images = os.listdir(dataset_dir + dir + "images/")
        for i in range(1, len(images)):
            old_name = images[i-1].split(".rf")[0]
            next_name = images[i].split(".rf")[0]
            label_name = images[i][:-3] + "txt"

            if old_name == next_name:
                os.remove(dataset_dir + dir + "images/" + images[i])
                os.remove(dataset_dir + dir + "labels/" + label_name)


remove_duplicates("./arabic_compiled_dataset/")


In [ ]:
!pip install yolov5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.4/953.4 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.1/871.1 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.9 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117030 sha

In [ ]:
import os

def renumber_classes(dataset_dir, class_map):
  dirs = ["train/", "test/", "valid/"]
  for dir in dirs:
    for label in os.listdir(dataset_dir + dir + "labels/"):
      with open(dataset_dir + dir + "labels/" + label, "r") as f:
        lines = f.readlines()
        for i in range(len(lines)):
          class_id = lines[i].split(" ")[0]
          class_id = class_map[int(class_id)]
          lines[i] = str(class_id) + " " + " ".join(lines[i].split(" ")[1:])

      with open(dataset_dir + dir + "labels/" + label, "w") as f:
        f.writelines(lines)

class_map = {
    0: 0,
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    5: 5,
    6: 6,
    7: 7,
    8: 8,
    9: 9,
    10: 10,
    12: 11,
    16: 12,
    20: 13,
    22: 14,
    23: 15,
    24: 16,
    25: 17,
    28: 18,
    29: 19,
    32: 20,
    36: 21,
    38: 22,
    39: 23,
    42: 24,
    43: 25,
    45: 26
}

renumber_classes("./arabic_compiled_dataset/", class_map)

In [ ]:
import os

class_count = {}
for label in os.listdir("./arabic_compiled_dataset/train/labels/"):
    with open("./arabic_compiled_dataset/train/labels/" + label, "r") as f:
        lines = f.readlines()
        for line in lines:
            class_id = line.split(" ")[0]
            if class_id not in class_count:
                class_count[class_id] = 1
            else:
                class_count[class_id] += 1

class_count

{'8': 3247,
 '2': 4130,
 '6': 3579,
 '3': 3382,
 '0': 4123,
 '4': 3405,
 '7': 3787,
 '10': 705,
 '1': 3535,
 '5': 3793,
 '25': 197,
 '16': 214,
 '12': 145,
 '9': 3076,
 '17': 90,
 '20': 169,
 '21': 84,
 '22': 302,
 '24': 164,
 '19': 253,
 '14': 143,
 '15': 164,
 '23': 240,
 '13': 169,
 '11': 103,
 '18': 78,
 '26': 34}

In [ ]:
import os

print(len(os.listdir("./arabic_compiled_dataset/train/images/")))
print(len(os.listdir("./arabic_compiled_dataset/test/images/")))
print(len(os.listdir("./arabic_compiled_dataset/valid/images/")))

7933
370
785


In [ ]:
!yolov5 train --weights yolov5s.pt --img 640 --epochs 30 --data ./arabic_ocr.yaml

In [ ]:
!yolov5 val --weights ./ocr_2.pt --data ./arabic_ocr.yaml --img 640

2024-08-27 18:31:43.761342: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-27 18:31:43.799551: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-27 18:31:43.811093: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
YOLOv5 🚀 2024-8-27 Python-3.10.12 torch-2.4.0+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7082944 parameters, 0 gradients, 16.0 GFLOPs
100% 755k/755k [00:00<00:00, 18.9MB/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
val: Scanning /content/arabic_compiled_d